In [1]:
####################### You have to run this cell #######################

% load_ext autoreload
% autoreload 2
import sys, os
import os
cwd = os.getcwd()
path = os.path.join(cwd, os.pardir)
sys.path.append(path)


# Put all imports below

In [2]:
from models import User,Test,Payments,UserRoles,Role,College,CollegeTest,QuestionAttempt,SectionAttempt,TestAttempt,Question,Choice,Section,CorporateApplicants
from models.Package import PackageTypes
from app import create_app
from extensions import db
from sqlalchemy import and_,outerjoin,or_
from sqlalchemy.orm import contains_eager, joinedload,load_only
from pprint import pprint
import json2html

In [7]:
app = create_app('./configs/prod.py')
app.app_context().push()

Exception during reset or similar
Traceback (most recent call last):
  File "/Users/harshvardhangupta/.virtualenvs/beatest-flask-rest-backend/lib/python3.6/site-packages/sqlalchemy/engine/base.py", line 704, in _rollback_impl
    self.engine.dialect.do_rollback(self.connection)
  File "/Users/harshvardhangupta/.virtualenvs/beatest-flask-rest-backend/lib/python3.6/site-packages/sqlalchemy/dialects/mysql/base.py", line 1775, in do_rollback
    dbapi_connection.rollback()
_mysql_exceptions.OperationalError: (2006, 'MySQL server has gone away')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/harshvardhangupta/.virtualenvs/beatest-flask-rest-backend/lib/python3.6/site-packages/sqlalchemy/pool.py", line 703, in _finalize_fairy
    fairy._reset(pool)
  File "/Users/harshvardhangupta/.virtualenvs/beatest-flask-rest-backend/lib/python3.6/site-packages/sqlalchemy/pool.py", line 871, in _reset
    self._reset_agent.rollback(

In [4]:
(TestAttempt.query
            .filter(TestAttempt.id == 10165)
            .join(CorporateApplicants,
                  and_(CorporateApplicants.user_id == TestAttempt.user_id,
                       CorporateApplicants.corporate_id == 1
                       )
                  )
            .join(SectionAttempt,
                  and_(SectionAttempt.test_attempt_id == TestAttempt.id,
                       SectionAttempt.id == 24659))
            .join(QuestionAttempt,
                  and_(QuestionAttempt.section_attempt_id == SectionAttempt.id,
                       QuestionAttempt.question_id == 7102))
             .options(contains_eager(TestAttempt.section_attempts)
                     .contains_eager(SectionAttempt.question_attempts))
            .one())

TestAttempt({
    "date": "2019-02-15T06:19:38",
    "focus_lost_count": 0,
    "id": 10165,
    "is_complete": true,
    "is_graded": true,
    "score": 0,
    "section_attempts": [
        {
            "correct_question_count": 0,
            "id": 24659,
            "incorrect_question_count": 0,
            "is_complete": false,
            "question_attempts": [
                {
                    "attempt_status": "seen",
                    "choice_id": null,
                    "chosen_language_id": 7,
                    "coding_average_time": null,
                    "coding_language": null,
                    "correct_coding_cases_count": 0,
                    "question_id": 7102,
                    "section_attempt_id": 24659,
                    "time_spent": 1725,
                    "tita_choice": null,
                    "was_correct": null
                }
            ],
            "score": 0,
            "section_id": 348,
            "test_attempt_id": 1016

In [ ]:
result = (TestAttempt.query
         .filter(TestAttempt.test_id==188)
         .join(TestAttempt.user)
          .join(TestAttempt.section_attempts)
          .join(SectionAttempt.question_attempts)
          .join(User.college)
         .options(contains_eager(TestAttempt.user).load_only(User.full_name,User.email)
                 .contains_eager(User.college).load_only(College.college_name))
          .options(contains_eager(TestAttempt.section_attempts).contains_eager(SectionAttempt.question_attempts))
         .all()
         )



In [ ]:
result = (TestAttempt.query
         .filter(TestAttempt.test_id==186)
         .join(TestAttempt.user)
          .join(TestAttempt.section_attempts)
          .join(SectionAttempt.question_attempts)
          .join(User.college)
         .options(contains_eager(TestAttempt.user).load_only(User.full_name,User.email)
                 .contains_eager(User.college).load_only(College.college_name))
          .options(contains_eager(TestAttempt.section_attempts).contains_eager(SectionAttempt.question_attempts))
         .all()
         )



In [ ]:
result

In [ ]:
t = TestAttempt.query.filter(TestAttempt.user_id==12420).one()

In [ ]:
TestAttempt.calculate_score_for_test(user_id=12404,test_id=186,should_persist=True)

In [ ]:
json.dumps([_.todict() for _ in result])

In [ ]:
json.dumps([_.todict() for _ in result])

## Simple Queries 
For very simple queries, use MODEL.query , followed by filter arguments 

In [ ]:
result = User.query.filter(User.id ==1).first()

pprint(result.todict(['full_name', # can specify which arguments to serialize
             'id',
             'referral_bonus',
             'referral_code',
             'wallet','college']) )

In [ ]:
db.session.close_all() # this is required for just this notebook 

In [ ]:
pprint(result)

## Queries with in clause 
To fetch all items that satisfy elements in a list

In [ ]:
result = User.query.filter(User.id.in_([1,2,3])).all()


pprint([result.todict(['full_name', # notice how this is serialised 
             'id',
             'referral_bonus',
             'referral_code',
             'wallet',
             'college']) for result in result])

In [ ]:
db.session.close_all() # this is required for just this notebook 

## Queries with nested relationships
Sometimes, queries with nested relationships are required. 
An example is User -> payments 

It is possible to fetch user first, then do user.payments  , but this will result in two queries. Also , it is not possible to do filtering on the nested resource. 

In [ ]:
# Get User with id =1 and all his payments 

result = db.session.query(User).filter(User.id==1).first()

In [ ]:
pprint(result.payments) # will do a 2nd query 

In [ ]:
db.session.close_all() # this is required for just this notebook 

## Better way of doing this

This will happen in one query. 
by adding a join and .options(contains_eager()), you are fooling sqlalchemy to think it has fetched all the things that it needs. 
Thus, calling test.payments will not cause a 2nd trip. 


In [ ]:
# Get User with id =1 and all his payments 
result = User.query\
        .join(User.payments)\
        .options(contains_eager(User.payments))\
        .filter(User.id ==1).all()[0]
        
# notice that I did not call first() , because first() will literally add 
#limit 1 to the end of the SQL query. In that case, sqlalchemy will be                                             #unable to parse the multi row result into a user -> multi payments object

db.session.close_all() # this is required for just this notebook 

In [ ]:
pprint(result.todict())

## Weird behaviour by `contains_eager()`

It is not possible to call `first()` in the previous example, because it will literally add `LIMIT 1`
to the query. This prevents sqlalchemy from parsing a potentially multi row result into a complex model object. 

However, calling `one()` does not explicitly add `LIMIT 1` to the query. It will still allow a multi row result to be returned , and parsed. 

The interesting thing is that , it will `one()` will restrict the primary object being searched for to exactly one object. 

It will throw an exception if multiple results for the primary object were returned

This somewhat helps when looking for only the first value. (it will still throw a multiple result exception if multiple rows for primary object is returned)



## Alternative, using joinedload

Calling first() on a joined query wil cause incorrect results, because sqlalchemy will literally add LIMIT 1 to the end of the query. 
In order to resolve this issue, there exists joinedload(). 
If joinedload is used instead of contains_eager, sqlalchemy first wraps the query in a sub query, and then applies LIMIT on that. 



In [ ]:
result = User.query\
        .options(joinedload(User.payments))\
        .filter(User.id ==1).first()

db.session.close_all() # this is required for just this notebook 

In [ ]:
pprint(result.todict(['payments',{'payments':'id'}]))

## JoinedLoad vs Contains Eager 

JoinedLoad is designed to be simple, and not have the ability to alter results using conditions. 
If you have to perform conditions on some joined table, joinedload will not be helpful. 

Contains_eager is meant to be a 'short_circuit' mechanism. It is your way of telling sqlalchemy that this is all the results I want. Use contains_eager when performing complex intermediate queries

# Warning!

When using contains_eager and having something like 

`some_query.all()[0]`

**Be extra careful of the query.** 
The query may be fetching millions of records , and eventually you are using just the first one. 
One way to prevent such mistakes is to always assert that the length of the result is 1. 

**The safer alternative is to use .one() instead **

## Conditional 

Adding a condition on the nested resource

In [ ]:
result = db.session.query(User) \
    .join(User.payments) \
    .options(contains_eager(User.payments)) \
    .filter(
        Payments.testId.in_([24, 58, 93, 4])) \
    .filter(User.id == 1).one_or_none()
# user.id =1 is guaranteed to have exactly 1 row in Users table, so one() should work. 
# if an exception occurs, theres a bigger problem and database admin should be notified.


In [ ]:
db.session.close_all() # this is required for just this notebook 

In [ ]:
pprint(result)

In [ ]:
pprint(result.payments) # will not call db again. 

## Multi Nesting 

The same logic can be extended to arbitrary amount of nesting. 

In this example , we want to get a particular user's payments , and the name of the test that those payments belong to. 


In [ ]:
# if you have multiple joins ,his is how you specify to load all(line 6-7)

result = db.session.query(User)\
        .join(User.payments)\
        .join(Payments.test)\
        .options(contains_eager(User.payments)\
                .contains_eager(Payments.test))\
        .filter(Payments.testId.in_([24,58,93,4]))\
        .filter(User.id ==1).all()[0] # notice this. same reason as previous example.
db.session.close_all() # this is required for just this notebook 

In [ ]:
pprint([result.test.name for result in result.payments])  # did not do a 2nd db call

## A Real Example

Check if a user can access a particular question attempt.<br>This is done by doing a join from 
TestAttempt --> SectionAttempts --> QuestionAttempts 

A filter to match the userid with the TestAttempt's user id , 
and the inputs for the (SectionAttempt.id , Question.id).<br>
If user has access to that particular pair, a row will be returned, else nothing 
will be returned.

In [ ]:
query = TestAttempt \
    .query \
    .join(SectionAttempt) \
    .join(QuestionAttempt) \
    .filter(QuestionAttempt.section_attempt_id == 12) \
    .filter(QuestionAttempt.question_id == 5689) \
    .filter(TestAttempt.user_id == 536) \
    .exists()  # this puts the rest of the query into a sub query, and adds exists() to the result

exists = db.session.query(query).scalar()
db.session.close_all()  # this is required for just this notebook


In [ ]:
pprint(exists)

## Tip - use load_only to really speed up queries

load_only prevents loading all columns. Sometimes, you dont need all the columns, and doing this<br>can really improve speed. E.g. By using only primary keys, its possible to speed up query speed by 10x.  

In many cases, especially joins , only ids are required. In such cases, you can notice a dramatic speed boost

### Example 

In order to calculate score, it is possible to do this join: 

`TestAttempt -> Test -> Sections -> Questions -> Choices`

and also

`TestAttempt -> Test -> Sections -> Questions -> QuestionAttempt`

Test has instruction html (which is lots of data), and so does Question.
We do not need these html in any way 

In [ ]:
def query_without_load_only():
    result = (TestAttempt.query
              .options(
            contains_eager(TestAttempt.test)
            .contains_eager(Test.sections)
            .contains_eager(Section.questions)
            .contains_eager(Question.choices))
              .options(
            contains_eager(TestAttempt.test)
            .contains_eager(Test.sections)
            .contains_eager(Section.questions)
            .contains_eager(Question.question_attempts))
              .join(TestAttempt.test)
              .join(Test.sections)
              .join(Section.questions)
              .join(Question.choices)
              .join(Question.question_attempts)
              .filter(TestAttempt.id == 1)
              .one())

    db.session.close_all()  # this is required for just this notebook 
    return result


In [ ]:
%timeit -n 1 query_without_load_only()

In [ ]:
pprint(query_without_load_only())

In [ ]:
def query_with_load_only():
    result = TestAttempt.query\
    .options(
        contains_eager(TestAttempt.test).load_only(Test.id)\
        .contains_eager(Test.sections).load_only(Section.id)\
        .contains_eager(Section.questions).load_only(
            Question.id,
            Question.points_correct,
            Question.points_wrong)\
            .contains_eager(Question.choices)\
                .load_only(Choice.id,Choice.is_correct)\
        )\
    .options(
        contains_eager(TestAttempt.test).load_only(Test.id)\
        .contains_eager(Test.sections).load_only(Section.id)\
        .contains_eager(Section.questions).load_only(Question.id)
        .contains_eager(Question.question_attempts)\
        )\
    .join(TestAttempt.test)\
    .join(Test.sections)\
    .join(Section.questions)\
    .join(Question.choices)\
    .join(Question.question_attempts)\
    .filter(TestAttempt.id == 1)\
    .one()
    
    db.session.close_all() # this is required for just this notebook 
    return result

In [ ]:
%timeit -n 1 query_with_load_only()

In [ ]:
pprint(query_with_load_only())

In [5]:
from Externals import BingSpellCheck

/Users/harshvardhangupta/.virtualenvs/beatest-flask-rest-backend/lib/python3.6/site-packages/sqlalchemy/ext/declarative/clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as models.QuestionAttempt.QuestionAttempt, and will be replaced in the string-lookup table.
  item.__name__
[autoreload of models.QuestionAttempt failed: Traceback (most recent call last):
  File "/Users/harshvardhangupta/.virtualenvs/beatest-flask-rest-backend/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/harshvardhangupta/.virtualenvs/beatest-flask-rest-backend/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 368, in superreload
    module = reload(module)
  File "/Users/harshvardhangupta/.virtualenvs/beatest-flask-rest-backend/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/Users/harshvardhangupt

In [10]:
BingSpellCheck.get_spelling_flagged_tokens("this is there dream")

cannot find the current segment/subsegment, please make sure you have a segment open
No segment found, cannot begin subsegment https://api.cognitive.microsoft.com/bing/v7.0/SpellCheck.
cannot find the current segment/subsegment, please make sure you have a segment open


[{'offset': 8,
  'token': 'there',
  'type': 'UnknownToken',
  'suggestions': [{'suggestion': 'the', 'score': 0.8486098721839048}]}]

#### Difference is 3.45 seconds vs 454 milliseconds. Over 7x faster! 